# Import Files

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving bid_data.csv to bid_data.csv


In [ ]:
import io
import pandas as pd
df2 = pd.read_csv(io.BytesIO(uploaded['bid_data.csv']))

# I. Exact Calculations

In [ ]:
import random

bids = df2.to_numpy()
values = [10,20,30,40,50,60,70,80,90,100]

bids1 = [9,10,20,30,40,55,54,53,52,51]
bids2 = [9,18,27,36,40,50,60,60,70,80]
our_bids = [bids1, bids2]
probabilities = [[],[]]
utilities = [[],[]]

for bidder in range(2):
  for col in range(10):
    bid = our_bids[bidder][col]
    v = values[col]
    count = 0
    total_utility = 0
    for row in bids:
      for bo in row:
        if bid > bo:
          count += 1
          total_utility += v - bid
        if bid == bo:
          if (random.randint(0,1) == 1):
              count += 1
              total_utility += v - bid
    p = count / (bids.shape[0] * bids.shape[1])
    u = total_utility/ (bids.shape[0] * bids.shape[1])
    probabilities[bidder].append(p)
    utilities[bidder].append(u)

probabilities_pd = pd.DataFrame(probabilities)
utilities_pd = pd.DataFrame(utilities)

In [ ]:
print("Exact calculation for winning probabilities:")
display(probabilities_pd)
print("Exact calculation for expected utilities:")
display(utilities_pd)

Exact calculation for winning probabilities:


,0,1,2,3,4,5,6,7,8,9
0,0.050000,0.107143,0.259524,0.378571,0.516667,0.709524,0.697619,0.697619,0.692857,0.676190
1,0.052381,0.219048,0.345238,0.473810,0.514286,0.640476,0.769048,0.769048,0.835714,0.895238


Exact calculation for expected utilities:


,0,1,2,3,4,5,6,7,8,9
0,0.050000,1.071429,2.595238,3.785714,5.166667,3.547619,11.161905,18.835714,26.328571,33.133333
1,0.052381,0.438095,1.035714,1.895238,5.142857,6.404762,7.690476,15.380952,16.714286,17.904762


# I. Monte Carlo

In [ ]:
import random

bids = df2.to_numpy()
values = [10,20,30,40,50,60,70,80,90,100]

bids1 = [9,10,20,30,40,55,54,53,52,51]
bids2 = [9,18,27,36,40,50,60,60,70,80]
our_bids = [bids1, bids2]
probabilities = [[],[]]
utilities = [[],[]]

N = 1000


In [ ]:
for bidder in range(2):
  for col in range(10):
    bid = our_bids[bidder][col]
    v = values[col]
    count = 0
    total_utility = 0
    for sample in range(N):
      i = random.randint(0, bids.shape[0]-1)
      j = random.randint(0, bids.shape[1]-1)
      bo = bids[i,j]
      if bid >= bo:
        count += 1
        total_utility += v - bid
      if bid == bo:
        if (random.randint(0,1) == 1):
          count += 1
          total_utility += v - bid

    p = count / N
    eu = total_utility/ N
    probabilities[bidder].append(p)
    utilities[bidder].append(eu)

mc_probabilities_pd = pd.DataFrame(probabilities)
mc_utilities_pd = pd.DataFrame(utilities)

In [ ]:
print("Monte Carlo calculation for winning probabilities:")
display(mc_probabilities_pd)
print("Monte Carlo calculation for expected utilities:")
display(mc_utilities_pd)

Monte Carlo calculation for winning probabilities:


,0,1,2,3,4,5,6,7,8,9
0,0.061,0.178,0.329,0.408,0.588,0.741,0.676,0.694,0.705,0.714
1,0.083,0.217,0.328,0.465,0.564,0.671,0.826,0.812,0.876,0.920


Monte Carlo calculation for expected utilities:


,0,1,2,3,4,5,6,7,8,9
0,0.061,1.780,3.290,4.08,5.88,3.705,10.816,18.738,26.79,34.986
1,0.083,0.434,0.984,1.86,5.64,6.710,8.260,16.240,17.52,18.400


# II. Exact Calculation (Brute Force)

In [ ]:
import numpy as np

values = [10,20,30,40,50,60,70,80,90,100]
optimal_bids = []
optimal_utilities = []

bids = df2.to_numpy()
probabilities = np.full_like(bids, 0)
utilities = np.full_like(bids, 0)

for v in values:
  optimal_eu = 0
  optimal_bid = 0
  for bi in range(v):
    count = 0
    total_utility = 0
    for row in bids:
      for bo in row:
        if bi > bo:
          count += 1
          total_utility += v - bi
        if bi == bo:
          if (random.randint(0,1) == 1):
              count += 1
              total_utility += v - bi
    p = count / (bids.shape[0] * bids.shape[1])
    eu = total_utility/ (bids.shape[0] * bids.shape[1])
    if eu > optimal_eu:
      optimal_eu = eu
      optimal_bid = bi
  optimal_bids.append(optimal_bid)
  optimal_utilities.append(optimal_eu)

utilities = pd.DataFrame(optimal_utilities)


In [ ]:
print("The exact calculation of optimal bids for each value are:")
display(utilities) 

The optimal bids for each value are:


,0
0,0.171429
1,1.523810
2,3.428571
3,5.790476
4,9.114286
5,12.566667
6,16.992857
7,21.821429
8,27.542857
9,33.573810


#II. Monte Carlo

In [ ]:
import random

bids = df2.to_numpy()
probabilities = np.full_like(bids, 0)
utilities = np.full_like(bids, 0)

N = 1000

In [ ]:
for r in range(bids.shape[0]):
  for c in range(bids.shape[1]):
    bi = bids[r][c]
    val = (c + 1) * 10
    count = 0
    total_utility = 0
    for sample in range(N):
      i = random.randint(0, bids.shape[0]-1)
      j = random.randint(0, bids.shape[1]-1)
      bo = bids[i,j]
      if bi >= bo:
        count += 1
        total_utility += val - bi
      if bi == bo:
        if (random.randint(0,1) == 1):
          count += 1
          total_utility += val - bi

      p = count / N
      eu = total_utility/ N
      probabilities[r, c] = p
      utilities[r, c] = eu

mc_probabilities_pd = pd.DataFrame(probabilities)
mc_utilities_pd = pd.DataFrame(utilities)

mc_max_utilities = mc_utilities_pd.max(0)
mc_max_utility_indexes = mc_utilities_pd.idxmax(0)
mc_index_list_max_u = mc_max_utility_indexes.tolist()
 
mc_optimum_bids = []
for i in range(10):
   mc_optimum_bids.append(bids[mc_index_list_max_u[i]][i])

[5.0, 10.0, 11.0, 20.0, 21.0, 30.0, 35.0, 40.0, 45.0, 50.0]

In [ ]:
print("The Monte Carlo calculation for optimal bids is")
display(mc_optimum_bids)

The Monte Carlo calculation for optimal bids is


[5.0, 10.0, 11.0, 20.0, 21.0, 30.0, 35.0, 40.0, 45.0, 50.0]

# Part 2: Bidding Algorithm

In [ ]:
# the input "bids" is a 2d array where each column represents values {10, 20, ..., 100} and each row represents a bidder's bids for each value


import numpy as np

bids = df2.to_numpy()
probabilities = np.full_like(bids, 0)
utilities = np.full_like(bids, 0)

def bidding_algorithm(bids):
  values = [10,20,30,40,50,60,70,80,90,100]
  optimal_bids = []
  optimal_utilities = []

  for v in values:
    optimal_eu = 0
    optimal_bid = 0
    for bi in range(v):
      count = 0
      total_utility = 0
      for row in bids:
        for bo in row:
          if bi > bo:
            count += 1
            total_utility += v - bi
          if bi == bo:
            if (random.randint(0,1) == 1):
                count += 1
                total_utility += v - bi
      p = count / (bids.shape[0] * bids.shape[1])
      eu = total_utility/ (bids.shape[0] * bids.shape[1])
      if eu > optimal_eu:
        optimal_eu = eu
        optimal_bid = bi
    optimal_bids.append(optimal_bid)
    optimal_utilities.append(optimal_eu)
  return optimal_bids

bidding_algorithm(bids)


[6, 11, 12, 21, 22, 31, 31, 41, 42, 52]